In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           Var,
                           Reals,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimPF6_PR import configuration
from R125_bmimPF6_PR import configuration as configuration_R125
from R32_bmimPF6_PR import configuration as configuration_R32

# Load data from csv
data = pd.read_csv('HFCSIL_all.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})
    
    m.fs.a = Var(within=Reals, bounds=(-6,6), initialize=-0.07)
    m.fs.b = Var(within=Reals, bounds=(-6,6), initialize=-0.02)
    m.fs.c = Var(within=Reals, bounds=(-6,6), initialize=0.4)
    m.fs.d = Var(within=Reals, bounds=(-6,6), initialize=0.06)

    eps1 = 0.05
    eps2 = 0.1
    if  float(data["x_R32"]) < 0.0001:

        m.fs.properties = GenericParameterBlock(default=configuration_R125)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data["x_R125"])+eps1
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp["bmimPF6"].fix(1-x)
        m.fs.state_block.mole_frac_comp["R125"].fix(x)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(1.6)
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.06)

        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].unfix()
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].unfix()
        
        # Fix the state variables on the state block
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
        m.fs.state_block.mole_frac_comp["R125"].unfix()
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "R125"].fix(float(data["x_R125"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"].fix(float(data["x_IL"]))
#         m.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"].unfix()
        m.fs.state_block.mole_frac_comp["R125"].fix(float(data["x_R125"])+eps1)
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()

        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A["bmimPF6", "R125"].setlb(-5)
        m.fs.properties.PR_kappa_A["bmimPF6", "R125"].setub(5)

        m.fs.properties.PR_kappa_A["R125", "bmimPF6"].setlb(-5)
        m.fs.properties.PR_kappa_A["R125", "bmimPF6"].setub(5)
        
        m.fs.A = Constraint(expr= m.fs.a == m.fs.properties.PR_kappa_A["bmimPF6", "R125"])
        m.fs.B = Constraint(expr= m.fs.b == m.fs.properties.PR_kappa_A["R125", "bmimPF6"])

    elif  float(data["x_R125"]) < 0.0001:

        m.fs.properties = GenericParameterBlock(default=configuration_R32)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data["x_R32"])+eps2
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp["bmimPF6"].fix(1-x)
        m.fs.state_block.mole_frac_comp["R32"].fix(x)

        # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.046599)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.37995)

        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].unfix()
        
        # Fix the state variables on the state block
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].unfix()
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "R32"].fix(float(data["x_R32"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"].fix(float(data["x_IL"]))
#         m.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"])+eps2)
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()

        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A["bmimPF6", "R32"].setlb(-5)
        m.fs.properties.PR_kappa_A["bmimPF6", "R32"].setub(5)

        m.fs.properties.PR_kappa_A["R32", "bmimPF6"].setlb(-5)
        m.fs.properties.PR_kappa_A["R32", "bmimPF6"].setub(5)
        
        m.fs.C = Constraint(expr= m.fs.c == m.fs.properties.PR_kappa_A["bmimPF6", "R32"])
        m.fs.D = Constraint(expr= m.fs.d == m.fs.properties.PR_kappa_A["R32", "bmimPF6"])

    else:

        m.fs.properties = GenericParameterBlock(default=configuration)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})

        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(298.15)
        m.fs.state_block.pressure.fix(100000)
        m.fs.state_block.mole_frac_comp["R32"].fix(0.02)
        m.fs.state_block.mole_frac_comp["R125"].fix(0.02)
        m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.96)

        m.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
        m.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.02)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.37995)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(1.6)
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.06)

   
        # Initialize the flash unit
        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        # Fix at actual temperature

        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["R125"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].unfix()
        m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
#         m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#         m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
        m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
        m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 

        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].unfix()
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].unfix()
        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].unfix()
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].unfix()
        
        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-10)
        m.fs.properties.PR_kappa_A['R32', 'R125'].setub(10)

        m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-10)
        m.fs.properties.PR_kappa_A['R125', 'R32'].setub(10)

        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-10)
        m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(10)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-10)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(10)

        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-10)
        m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(10)

        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-10)
        m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(10)
        
        m.fs.A = Constraint(expr= m.fs.a == m.fs.properties.PR_kappa_A["bmimPF6", "R125"])
        m.fs.B = Constraint(expr= m.fs.b == m.fs.properties.PR_kappa_A["R125", "bmimPF6"])
        m.fs.C = Constraint(expr= m.fs.c == m.fs.properties.PR_kappa_A["bmimPF6", "R32"])
        m.fs.D = Constraint(expr= m.fs.d == m.fs.properties.PR_kappa_A["R32", "bmimPF6"])
        
        # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.000001, "x_IL":0.7312,  
             "x_R125":0.5, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2022-02-24 15:18:31 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
The initial DOF is 2


In [7]:
variable_name = [
                "fs.a",
                "fs.b",
                "fs.c",
                "fs.d"]

In [8]:
# def SSE(m, data):
#     expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
#     return expr*1e-9

def SSE(m, data):
    if  float(data["x_R32"]) < 0.0001:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9
    elif  float(data["x_R125"]) < 0.0001:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9
    else:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9*10*2.62

In [9]:
solver_options = {"max_iter": 10}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2022-02-24 15:18:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:18:48 [INFO] idaes

2022-02-24 15:19:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-24 15:19:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technica

  66  2.3920469e+00 2.23e+00 5.61e+05  -8.6 5.35e+03   0.3 9.80e-02 8.44e-02h  1
  67r 2.3920469e+00 2.23e+00 9.99e+02   0.3 0.00e+00  -0.1 0.00e+00 2.00e-10R  2
  68r 2.3920467e+00 2.29e+00 1.00e+03   0.3 2.54e+09    -  4.56e-09 1.89e-10f  3
  69r 2.3919045e+00 2.08e+00 1.00e+03   0.3 1.64e+05    -  3.16e-04 2.99e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70r 2.3918708e+00 7.30e-01 1.67e+03   0.3 1.03e+04    -  5.52e-02 3.25e-03f  1
  71  2.2757252e+00 4.75e-01 3.83e-01  -8.6 5.95e+04    -  8.14e-01 7.07e-01h  1
  72  2.4772903e+00 1.13e+00 2.80e+00  -8.6 4.67e+04    -  9.84e-01 1.00e+00h  1
  73  2.3334223e+00 6.26e-01 2.36e+00  -8.6 6.10e+04    -  1.53e-01 4.48e-01h  1
  74  2.2637236e+00 7.09e-01 1.20e+00  -8.6 3.38e+04    -  7.23e-01 1.00e+00f  1
  75  2.3232950e+00 5.67e-01 1.59e+00  -8.6 6.15e+04    -  8.60e-01 5.00e-01h  2
  76  2.2992445e+00 4.45e-01 1.25e+00  -8.6 3.07e+04    -  1.00e+00 2.53e-01h  1
  77  2.4856685e+00 2.35e-02

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 2.484321

The values for the parameters are as follows:
fs.a = 0.41376702208116806
fs.b = 0.05946006640846661
fs.c = -0.11305005887608774
fs.d = -0.03283152442120816


In [11]:
print(large_residuals_set(pest.ef_instance))
# print(degrees_of_freedom(pest.ef_instance))

ComponentSet([])


In [12]:
# fs.properties.PR_kappa_A[R32,bmimPF6] = -0.03213185866632573
# fs.properties.PR_kappa_A[bmimPF6,R32] = -0.15261951215083822
# fs.properties.PR_kappa_A[bmimPF6,R125] = 0.4104779938348984
# fs.properties.PR_kappa_A[R125,bmimPF6] = 0.05514778922730282

In [13]:
from idaes.core.util.model_statistics import report_statistics
print(report_statistics(pest.ef_instance))


Model Statistics  -  _Q_opt 

Degrees of Freedom: 10 

Total No. Variables: 3912 
    No. Fixed Variables: 2634
    No. Unused Variables: 1089 (Fixed):1089)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 1268 
    No. Equality Constraints: 1268 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 70 (Deactivated: 69)

No. Blocks: 561 (Deactivated: 0) 
No. Expressions: 2475 

None
